Para los datos del estado de mexico que son historicos se tiene que realizar lo siguiente:

*  Eliminar columnas: Precipitacion, Evaporacion.
*  Se la temperatura maxima y minima en algun registro es nulo eliminarlo.
*  Si es nulo  alguno de los valores de temperatura, calcular el promedio con las dos temperaturas anteriores y las dos temperaturas posteriores.


In [ ]:
pip install azure-storage-blob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from azure.storage.blob import BlobServiceClient
from io import BytesIO
from google.colab import userdata
secret_connection_string = userdata.get('connection_string')
import os

# Cadena de conexión al almacenamiento de Blob de Azure
connection_string = secret_connection_string;

# Nombre del contenedor
container_name = "diplomadodc"

folder_name = "data"
# Crea el cliente del servicio de Blob
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Obtén una referencia al contenedor
container_client = blob_service_client.get_container_client(container_name)

# Lista los blobs dentro del contenedor
blobs = container_client.list_blobs(name_starts_with=folder_name)

In [ ]:
dataframes = {}

# Itera sobre los blobs y crea un DataFrame por cada archivo
for blob in blobs:
    # Obtiene el nombre del archivo
    file_name = blob.name

    # Descarga los datos del archivo
    blob_data = container_client.download_blob(file_name)

    # Lee los datos del archivo y crea un DataFrame
    with BytesIO() as bio:
        blob_data.download_to_stream(bio)
        bio.seek(0)
        df = pd.read_excel(bio)  # Lee el archivo como Excel

    # Agrega el DataFrame al diccionario usando el nombre del archivo como clave
    dataframes[file_name] = df


In [ ]:
dataframes['data/Atlacomulco_utf8.xlsx']

KeyError: 'data/Atlacomulco_utf8.xlsx'

In [ ]:
import requests
import json

# URL del archivo JSON
url = "https://practicas6336904464.blob.core.windows.net/diplomadodc/jsonData.json"

try:
    # Obtener el contenido de la URL
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa (código 200)
    if response.status_code == 200:
        # Parsear el contenido JSON
        data = response.json()

        # Ahora puedes acceder a los datos como un diccionario
        print(data)

    else:
        print("Error al obtener el archivo:", response.status_code)

except Exception as e:
    print("Error:", e)




{'edoMex': [{'tipo': '3', 'titulo': 'nombre_estacion', 'nombre': 'VALLE DE BRAVO', 'url': 'https://practicas6336904464.blob.core.windows.net/diplomadodc/data/Valle_utf8.xlsx'}, {'tipo': '3', 'titulo': 'nombre_estacion', 'nombre': 'ATLACOMULCO', 'url': 'https://practicas6336904464.blob.core.windows.net/diplomadodc/data/Atlacomulco_utf8.xlsx'}, {'tipo': '3', 'titulo': 'nombre_estacion', 'nombre': 'Toluca', 'url': 'https://practicas6336904464.blob.core.windows.net/diplomadodc/data/Toluca_utf8.xlsx'}, {'tipo': '3', 'titulo': 'nombre_estacion', 'nombre': 'NEVADO DE TOLUCA', 'url': 'https://practicas6336904464.blob.core.windows.net/diplomadodc/data/nevado.xlsx'}, {'tipo': '3', 'titulo': 'nombre_estacion', 'nombre': 'LAGUNAS DE ZEMPOALA', 'url': 'https://practicas6336904464.blob.core.windows.net/diplomadodc/data/laguna.xlsx'}, {'tipo': '3', 'titulo': 'nombre_estacion', 'nombre': 'CERRO CATEDRAL', 'url': 'https://practicas6336904464.blob.core.windows.net/diplomadodc/data/cerro.xlsx'}, {'tipo':

In [ ]:
files = os.listdir()

# Sube cada archivo al Blob Storage
for file_name in files:
    # Ignora los archivos que no son archivos regulares (por ejemplo, directorios)
    if not os.path.isfile(file_name):
        continue

    # Sube el archivo al Blob Storage
    with open(file_name, "rb") as data:
        #blob_client = container_client.upload_blob(file_name, data)
        blob_client = container_client.upload_blob(folder_name + "/" + file_name, data)

    print(f"Archivo {file_name} subido correctamente al contenedor {container_name}.")

NameError: name 'os' is not defined

In [ ]:
# prompt: quiero ejecutar una peticion http tipo post

import requests
import json

url = "https://processing-file.azurewebsites.net/api/http_trigger"


# Datos del cuerpo en formato JSON
body = {"name": "Cristian"}

# Realiza la solicitud POST con el cuerpo JSON
response = requests.post(url, json=body)

# Imprime la respuesta
print(response.text)


Hello, Cristian. This HTTP triggered function executed successfully.


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import chardet

import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import os

# Autenticación
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

# Crear instancia del cliente de PyDrive
drive = GoogleDrive(gauth)

# ID de la carpeta compartida que deseas listar
carpeta_id = '1pvA46f64RB_3CpOXZQ68iMGgWBZkkqs-'

# Lista para almacenar los nombres de los archivos
files_dataframes = []

# Obtener la lista de archivos en la carpeta compartida
file_list = drive.ListFile({'q': "'{}' in parents".format(carpeta_id)}).GetList()

for archivo in file_list:
    try:
        # Descargar el archivo y guardarlo en el directorio actual
        archivo.GetContentFile(archivo['title'])
        # Leer el archivo descargado y crear un DataFrame
        df = pd.read_excel(archivo['title'])
        # Agregar el DataFrame al array
        files_dataframes.append(df)
        print("Archivo descargado y DataFrame creado:", archivo['title'])
    except Exception as e:
        print("Error al descargar y crear DataFrame para el archivo:", archivo['title'])
        print(e)

Archivo descargado y DataFrame creado: Edo Amecameca.xlsx
Archivo descargado y DataFrame creado: PueblaSan Nicolas.xlsx
Archivo descargado y DataFrame creado: laguna.xlsx
Archivo descargado y DataFrame creado: cerro.xlsx
Archivo descargado y DataFrame creado: nevado.xlsx


            Fecha Precipitacion Evaporacion TemperaturaMaxima  \
0      14/03/1969      0            Nulo              Nulo      
1      15/03/1969      0            Nulo              22.5      
2      16/03/1969      0            Nulo              22        
3      17/03/1969      4.3          Nulo              13.5      
4      18/03/1969      1.5          Nulo              18.5      
...           ...           ...         ...               ...   
13644  28/05/2021      5.5          Nulo              24        
13645  29/05/2021      1            Nulo              23        
13646  30/05/2021      5            Nulo              23        
13647  31/05/2021      Nulo         Nulo              Nulo      
13648  ----------      --------     -------           -------   

      TemperaturaMinima  
0                  Nulo  
1                  7.5   
2                  7     
3                  6     
4                  11.5  
...                 ...  
13644              12    
13645      